# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [62]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt
import hvplot.pandas

# Import API key
from api_keys import geoapify_key

In [63]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,19.20,69,43,6.14,PN,1722912307
1,1,waitangi,-43.9535,-176.5597,11.57,79,9,2.24,NZ,1722912308
2,2,udachny,66.4167,112.4000,10.77,67,55,11.70,RU,1722912309
3,3,chonchi,-42.6128,-73.8092,6.18,96,100,1.90,CL,1722912310
4,4,fort mcmurray,56.7268,-111.3810,19.77,59,0,2.06,CA,1722912311


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [64]:
%%capture --no-display

# Display the map
# YOUR CODE HERE
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size=50,
    color='City',
    hover_cols=["City"]
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [65]:

# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data with specific columns to match the provided image
ideal_weather_df = ideal_weather_df[['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Country', 'Date']]

# Display the first 20 rows to match the provided image
ideal_weather_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,remire-montjoly,4.9167,-52.2667,25.02,100,0,0.00,GF,1722912328
22,22,fort bragg,35.1390,-79.0060,25.08,87,0,2.57,US,1722912330
25,25,belmonte,-15.8631,-38.8828,21.22,77,0,1.99,BR,1722912333
108,108,camiri,-20.0500,-63.5167,21.45,71,0,2.42,BO,1722912422
199,199,clarksburg,39.2807,-80.3445,24.39,89,0,0.00,US,1722912522
208,208,rio preto da eva,-2.6994,-59.6997,25.42,47,0,0.91,BR,1722912533
242,242,meadow lakes,61.6247,-149.6011,22.62,40,0,3.60,US,1722912571
261,261,mount isa,-20.7333,139.5000,23.87,41,0,4.12,AU,1722912592
279,279,marand,38.4329,45.7749,23.92,50,0,0.75,IR,1722912612
299,299,santa vitoria,-18.8386,-50.1214,21.81,43,0,3.23,BR,1722912633


### Step 3: Create a new DataFrame called `hotel_df`.

In [66]:

# Display sample data with specific columns to match the provided image
ideal_weather_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
20,remire-montjoly,GF,4.9167,-52.2667,100,
22,fort bragg,US,35.1390,-79.0060,87,
25,belmonte,BR,-15.8631,-38.8828,77,
108,camiri,BO,-20.0500,-63.5167,71,
199,clarksburg,US,39.2807,-80.3445,89,
208,rio preto da eva,BR,-2.6994,-59.6997,47,
242,meadow lakes,US,61.6247,-149.6011,40,
261,mount isa,AU,-20.7333,139.5000,41,
279,marand,IR,38.4329,45.7749,50,
299,santa vitoria,BR,-18.8386,-50.1214,43,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [67]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()


    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
remire-montjoly - nearest hotel: Complexe Belova
fort bragg - nearest hotel: Airborne Inn Lodging
belmonte - nearest hotel: No hotel found
camiri - nearest hotel: Sol Chaqueño
clarksburg - nearest hotel: Parsons Hotel
rio preto da eva - nearest hotel: Hotel Santo Antonio
meadow lakes - nearest hotel: Best Western Lake Lucille Inn
mount isa - nearest hotel: Ibis Styles
marand - nearest hotel: No hotel found
santa vitoria - nearest hotel: hotel central
sosnovo-ozerskoye - nearest hotel: Озёрная
lewistown - nearest hotel: No hotel found
coxim - nearest hotel: Avendita
danville - nearest hotel: Holiday Inn Express Danville
hohhot - nearest hotel: Yue Jia Budget Hotel (foreigners approved)
la foa - nearest hotel: Oasis de Tendea
maua - nearest hotel: Go Inn Santo André
argo - nearest hotel: Apollon Hotel
juneau - nearest hotel: Ramada by Wyndham Juneau
paphos - nearest hotel: Mayfair Gardens Hotel
west plains - nearest hotel: Hampton


,City,Country,Lat,Lng,Humidity,Hotel Name
20,remire-montjoly,GF,4.9167,-52.2667,100,Complexe Belova
22,fort bragg,US,35.1390,-79.0060,87,Airborne Inn Lodging
25,belmonte,BR,-15.8631,-38.8828,77,No hotel found
108,camiri,BO,-20.0500,-63.5167,71,Sol Chaqueño
199,clarksburg,US,39.2807,-80.3445,89,Parsons Hotel
208,rio preto da eva,BR,-2.6994,-59.6997,47,Hotel Santo Antonio
242,meadow lakes,US,61.6247,-149.6011,40,Best Western Lake Lucille Inn
261,mount isa,AU,-20.7333,139.5000,41,Ibis Styles
279,marand,IR,38.4329,45.7749,50,No hotel found
299,santa vitoria,BR,-18.8386,-50.1214,43,hotel central


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [68]:
%%capture --no-display

# Display the map
# YOUR CODE HERE
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size=50,
    color='City',
    hover_cols=["City", "Country", "Hotel Name", "Humidity"]
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name,Humidity)